In [ ]:
import pandas as pd
from tqdm import tqdm
import pickle
from ray.tune import Analysis, ExperimentAnalysis

In [ ]:
import sys
sys.path.append("/p/project/raise-ctp2/cern/particleflow/mlpf")
from utils import count_skipped_configurations

In [ ]:
ray_results_folder = "/p/project/raise-ctp2/cern/ray_results/"  # Main folder containing all ray experiments

# Which experiment to analyze
exp_dir = ray_results_folder + "delphes_svr_dataset"

In [ ]:
expanalysis = ExperimentAnalysis(exp_dir, default_metric="val_loss", default_mode="min")
result_df = expanalysis.dataframe()

In [ ]:
hp_df = pd.concat([result_df["trial_id"], result_df.filter(regex="config/*")], axis=1)
hp_df

In [ ]:
learning_curves = {}
for logdir, trial_df in tqdm(expanalysis.trial_dataframes.items(), total=len(expanalysis.trial_dataframes)):
    trial_id = trial_df["trial_id"].values[0]
    learning_curves.update({trial_id: [trial_df, hp_df[hp_df["trial_id"] == trial_id]]})

In [ ]:
list(learning_curves.keys())[:6]

In [ ]:
example_trial = learning_curves['8275d_00122']
type(example_trial), len(example_trial)

In [ ]:
print('Trial shape:', example_trial[0].shape)
example_trial[0].head(3)

In [ ]:
example_trial[1]

In [ ]:
with open('LC_data.pkl', 'wb') as f:
    pickle.dump(learning_curves, f)